In [80]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [89]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [93]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [94]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.load(line.strip())
            yield line.strip()

In [95]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [96]:
def save_report(file_path, df):
    print(df)
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)

In [97]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [98]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(5):
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

Prompt==> {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-de100cdac55641b8b7cd4c941ca9d03e","object":"text_completion","created":435802,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef reverse_string(s):\n    return s[::-1]\n\nprint(reverse_string(\"Hello World\"))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":68,"completion_tokens":32}}
Execution time: 1.929948 seconds 


Prompt==> {"payload":"Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome."} 
 {"id":"cmpl-26689970b99745ebb295832828b4e25c","object":"text_completion","created":435802,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices"

Prompt==> {"payload":"Problem: Write a function in java to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s."} 
 {"id":"cmpl-2e3560519a0546158c63c3ae477be5f9","object":"text_completion","created":435810,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":42,"completion_tokens":1}}
Execution time: 0.111196 seconds 


Prompt==> {"payload":"Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-479b09acb6d543f181deee54cac330dc","object":"text_completion","created":435810,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprob

Prompt==> {"payload":"Problem: Write a function in c++ to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."} 
 {"id":"cmpl-f854ae7744014e289e1efeb628d427a7","object":"text_completion","created":435811,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nHere is a simple C++ function to check if a given number is prime:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nbool isPrime(int num) {\n    if (num <= 1) {\n        return false;\n    }\n    for (int i = 2; i * i <= num; i++) {\n        if (num % i == 0) {\n            return false;\n        }\n    }\n    return true;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a number: \";\n    cin >> num;\n    if (isPrime(num)) {\n        cout << num << \" is a prime number.\" << endl;\n    } else {\n        cout << num << \" is not a prime number.\" << endl;\n    }\n  

Prompt==> {"payload":"Problem: Write a function in c++ to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-50a402cf39314df08d0c62b3e1a39339","object":"text_completion","created":435820,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n#include <iostream>\n#include <vector>\n\nint binarySearch(std::vector<int>& arr, int target) {\n    int left = 0;\n    int right = arr.size() - 1;\n\n    while (left <= right) {\n        int mid = left + (right - left) / 2;\n\n        if (arr[mid] == target)\n            return mid;\n\n        if (arr[mid] < target)\n            left = mid + 1;\n        else\n            right = mid - 1;\n    }\n\n    return -1;\n}\n\nint main() {\n    std::vector<int> arr = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};\n    int target = 5;\n\n    int","logprobs":null,"fin

Prompt==> {"payload":"Problem: Write a function in Golang to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."} 
 {"id":"cmpl-5b2c64328fb847a19c68e8259bad5c3e","object":"text_completion","created":435836,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":44,"total_tokens":45,"completion_tokens":1}}
Execution time: 0.125170 seconds 


Prompt==> {"payload":"Problem: Write a function in Golang to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome."} 
 {"id":"cmpl-9bbc7e8db9804cc095b87fe335e8a521","object":"text_completion","created":435836,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reas

Prompt==> {"payload":"Problem: Implement a function in CSS to validate an email address.Input: String, email address.Output: Boolean, indicating whether the email address is valid."} 
 {"id":"cmpl-7db0ca72f8ad41c0b5cc8930c54db9f1","object":"text_completion","created":435837,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":41,"completion_tokens":1}}
Execution time: 0.125809 seconds 


Prompt==> {"payload":"Problem: Create a function in CSS to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element)."} 
 {"id":"cmpl-7f8edfc4330c41008eb37a0f1d5238a8","object":"text_completion","created":435837,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[

Prompt==> {"payload":"Problem: Implement form validation in angular to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid."} 
 {"id":"cmpl-c00cf5ed9c214e68b83734418c6908ed","object":"text_completion","created":435838,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}
Execution time: 0.124337 seconds 


Prompt==> {"payload":"Problem: Create an image slider in angular that displays images in a sequence with next and previous buttons.Input: Array of image URLs.Output: None (update the displayed image)."} 
 {"id":"cmpl-7ebc179e29cc4fd98196352a8b5bf3be","object":"text_completion","created":435838,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices

Prompt==> {"payload":"Problem: Create a function in jQuery to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element)."} 
 {"id":"cmpl-c7c8ffc316884a95bd875a05b45118ad","object":"text_completion","created":435839,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":50,"total_tokens":51,"completion_tokens":1}}
Execution time: 0.113886 seconds 


Prompt==> {"payload":"Problem: Write a function in Ruby to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams."} 
 {"id":"cmpl-b74083bde9ae4f2ea08f43ca322ecca3","object":"text_completion","created":435833,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))